In [ ]:
请你阅读并学习下面两段代码：
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.io as pio
import os
import numpy as np

text = "randomcase"

# 定义文件夹路径和CSV文件列表
data_folder = os.path.join('Data', text)
files = ['200_10_smallest.csv','200_10_cluster.csv','200_10_dynamic.csv']
curve_names = ['smallest','cluster', 'dynamic']
colors = px.colors.qualitative.Plotly[:len(curve_names)] 

# 创建三个DataFrame分别存储原始均值、最优均值和累积遗憾
all_mean_data = pd.DataFrame()
all_best_data = pd.DataFrame()
all_cumsum_data = pd.DataFrame()  # 新增：存储Cumulative Regret

# 循环处理每个文件
for file, name in zip(files, curve_names):
    file_path = os.path.join(data_folder, file)
    df = pd.read_csv(file_path, index_col='Repetition')
    
    # 计算每个step的均值（Instant Regret）
    mean_results = df.mean(axis=0)
    
    # 计算当前最优均值（Simple Regret，可选保留或删除）
    best_results = np.minimum.accumulate(mean_results.values)
    
    # 添加到原始均值DataFrame
    temp_mean_df = pd.DataFrame({
        'Iter': range(1, 151),
        'Value': mean_results.values,
        'Condition': name
    })
    
    # 添加到最优均值DataFrame（可选保留或删除）
    temp_best_df = pd.DataFrame({
        'Iter': range(1, 151),
        'Value': best_results,
        'Condition': name
    })    
    
    # 新增：计算Cumulative Regret（排除Uniform条件）
    if name != 'Uniform':
        cumsum_results = np.cumsum(mean_results.values)
        temp_cumsum_df = pd.DataFrame({
            'Iter': range(1, 151),
            'Value': cumsum_results,
            'Condition': name
        })
        all_cumsum_data = pd.concat([all_cumsum_data, temp_cumsum_df])
    
    all_mean_data = pd.concat([all_mean_data, temp_mean_df])
    all_best_data = pd.concat([all_best_data, temp_best_df])

# 创建子图（2行1列，垂直排列）
fig = make_subplots(
    rows=2, cols=1,
    vertical_spacing=0.1,
    subplot_titles=("Instant Regret", "Cumulative Regret")  # 添加子图标题
)

# 为每个条件添加原始均值曲线（Instant Regret，第一个子图）
for i, condition in enumerate(curve_names):
    mean_df = all_mean_data[all_mean_data['Condition'] == condition]
    fig.add_trace(
        go.Scatter(
            x=mean_df['Iter'],
            y=mean_df['Value'],
            name=condition,
            legendgroup=condition,
            showlegend=True,
            line=dict(color=colors[i]),
            meta=[condition],
            hovertemplate="Iter: %{x}<br>Mean: %{y:.4f}<br>Condition: %{meta[0]}<extra></extra>"
        ),
        row=1, col=1
    )

# 为每个条件（排除Uniform）添加累积遗憾曲线（Cumulative Regret，第二个子图）
for i, condition in enumerate(curve_names):
    if condition == 'Uniform':
        continue  # 跳过Uniform条件
    cumsum_df = all_cumsum_data[all_cumsum_data['Condition'] == condition]
    fig.add_trace(
        go.Scatter(
            x=cumsum_df['Iter'],
            y=cumsum_df['Value'],
            name=condition,
            legendgroup=condition,
            showlegend=False,  # 避免重复图例
            line=dict(color=colors[i]),
            meta=[condition],
            hovertemplate="Iter: %{x}<br>CumSum: %{y:.4f}<br>Condition: %{meta[0]}<extra></extra>"
        ),
        row=2, col=1
    )

# 更新布局
fig.update_layout(
    title_text=text+"_200_10",
    title_x=0.5,
    height=600,
    width=800,
    template='plotly_white',
    legend_title_text='Condition',
    legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.95)
)

# 更新Y轴和X轴标签
fig.update_yaxes(title_text="Instant Regret", row=1, col=1)
fig.update_yaxes(title_text="Cumulative Regret", row=2, col=1)
fig.update_xaxes(title_text="Iteration", row=2, col=1)  # 仅底部子图显示X轴标签

# 显示图形
fig.show()

# 保存为HTML文件
#pio.write_html(fig, '200_10_instant.jpg')

In [ ]:
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import os
import plotly.io as pio

# ---------- 配置 ----------
data_folder = os.path.join('Data', text) # 根据你的data变量自动设置

# 新旧算法文件列表（假设curve_names相同）
old_files = ['200_10_smallest_pre.csv','200_10_cluster_pre.csv', '200_10_dynamic_pre.csv']
new_files = ['200_10_smallest_post.csv','200_10_cluster_post.csv', '200_10_dynamic_post.csv']
colors = px.colors.qualitative.Plotly[:len(old_files)]  # 统一颜色

# ---------- 数据加载函数 ----------
def load_data(files):
    df_all = pd.DataFrame()
    for file, name in zip(files, curve_names):
        df = pd.read_csv(os.path.join(data_folder, file), index_col='Repetition')
        mean = df.mean(axis=0)
        temp_df = pd.DataFrame({
            'Iter': range(1, 151),
            'Value': mean.values,
            'Condition': name
        })
        df_all = pd.concat([df_all, temp_df])
    return df_all

# 加载新旧算法数据
old_data = load_data(old_files)
new_data = load_data(new_files)

# ---------- 绘图 ----------
fig = make_subplots(
    rows=2, cols=1,
    subplot_titles=("Obs-based", "Global-based"),
    vertical_spacing=0.1
)

# 绘制原始算法（第一子图）
for i, name in enumerate(curve_names):
    df = old_data[old_data['Condition'] == name]
    fig.add_trace(
        go.Scatter(
            x=df['Iter'], y=df['Value'],
            name=name, line=dict(color=colors[i]),
            showlegend=True,
            hovertemplate="Iter: %{x}<br>Value: %{y:.4f}<br>Condition: %{meta[0]}<extra></extra>",
            meta=[name]
        ),
        row=1, col=1
    )

# 绘制新算法（第二子图）
for i, name in enumerate(curve_names):
    df = new_data[new_data['Condition'] == name]
    fig.add_trace(
        go.Scatter(
            x=df['Iter'], y=df['Value'],
            name=name, line=dict(color=colors[i]),
            showlegend=False,  # 避免图例重复
            hovertemplate="Iter: %{x}<br>Value: %{y:.4f}<br>Condition: %{meta[0]}<extra></extra>",
            meta=[name]
        ),
        row=2, col=1
    )

# ---------- 更新布局 ----------
fig.update_layout(
    title_text=f"{text}_200_10",
    title_x=0.5,
    height=600,                    
    width=800, 
    template='plotly_white',
    legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.95)
)

# 统一Y轴标签
fig.update_yaxes(title_text="Simple Regret", row=1, col=1)
fig.update_yaxes(title_text="Simple Regret", row=2, col=1)
fig.update_xaxes(title_text="Iteration", row=2, col=1)

fig.show()
#pio.write_html(fig, '200_10_simple.jpg')